### 日期识别

In [1]:
import re
from datetime import datetime,timedelta
from dateutil.parser import parse
import jieba.posseg as psg

In [52]:
def time_extract(text):
    #存储每一个时间点
    time_res=[]
    #暂存时间
    word=''
    #时间映射到数字
    keyDate={'今天':0,'明天':1,'后天':2}
    #用jieba进行分词，k为得到的词语，v是词性
    for k,v in psg.cut(text):
        #判断词语是否在keydate中存在
        if k in keyDate:
            #如果word不为空，则在列表中添加相应的词语
            if word!='':
                time_res.append(word)
            #根据当前时间处理句子中出现的时间跨度，当前时间和时间跨度相加，格式化为x年x月x日的格式
            word=(datetime.today()+timedelta(days=keyDate.get(k,0))).strftime('%Y{0}%m{1}%d{2}'.format('年','月','日'))
        #如果word不为空，
        elif word!='':
            #如果得到的词性是m或t（分别代表数字和时间）
            if v in ['m','t']:
                #在word后追加此词
                word=word+k
            else:
                #如果后面跟的不是数字或时间，就把当前word存入time_res，就相当于这个时间段已经被找到，开始找下一个时间段
                time_res.append(word)
                word=''
        #如果找到的词性是数字或时间，且word为空，就直接存入
        elif v in ['m','t']:
            word=k
    if word!='':
        time_res.append(word)
    
    result=list(filter(lambda x:x is not None,[check_time_valid(w) for w in time_res]))
    final_res=[parse_datetime(w) for w in result]
    return [x for x in final_res if x is not None]   

#数据清洗，把号/日全部换成日，递归处理
def check_time_valid(word):
    m=re.match('\d+$',word)
    if m:
        if len(word)<=6:
            return None
    word1=re.sub('[号|日]\d+$','日',word)
    if word1!=word:
        return check_time_valid(word1)
    else:
        return word1

#将日期格式化成datetime的时间，
def parse_datetime(msg):
    if msg is None or len(msg)==0:
        return None
    try:
        dt=parse(msg)
        return dt.strftime(' %Y-%m-%d %H:%M:%S')
    except Exception as e:
        m = re.match(r"([0-9零一二两三四五六七八九十]+年)?([0-9一二两三四五六七八九十]+月)?([0-9一二两三四五六七八九十]+[号日])?([上中下午晚早]+)?([0-9零一二两三四五六七八九十百]+[点:.时])?([0-9零一二三四五六七八九十百]+分)?([0-9零一二三四五六七八九十百]+秒)?", msg)
        #group(0)就是匹配正则表达式的整体结果，大于0就是返回一个包含所有小组字符串的元组，从 1 到 所含的小组号。
        if m.group(0) is not None:
            res={
                "year":m.group(1),
                "month":m.group(2),
                "day":m.group(3),
                "hour":m.group(5) if m.group(5) is not None else '00',
                "minute":m.group(6) if m.group(6) is not None else '00',
                "second":m.group(7) if m.group(7) is not None else '00'
            }
            params={}
            #循环处理res
            for name in res:
                if res[name] is not None and len(res[name])!=0:
                    tmp=None
                    #如果是年的话调用year2dig进行处理
                    if name=='year':
                        tmp=year2dig(res[name][:-1])
                    #其他时间就进入cn2dig处理
                    else:
                        tmp=cn2dig(res[name][:-1])
                    if tmp is not None:
                        params[name]=int(tmp)
            #将今天的日期替换成params的内容，双星号（**）将参数以字典的形式导入
            target_date=datetime.today().replace(**params)
            #判断是否为下午，然后转化成24小时制
            is_pm=m.group(4)
            if is_pm is not None:
                if is_pm==u'下午' or is_pm==u'晚上' or is_pm=='中午':
                    hour=target_date.time().hour
                    if hour<12:
                        target_date=target_date.replace(hour=hour+12)
            return target_date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return None
        
UTIL_CN_NUM={'零':0,'一':1,'二':2,'三':3,'四':4,'五':5,'六':6,'七':7,'八':8,'九':9,'十':10,'两':2,'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9}
UTIL_CN_UNIT={'十':10,'百':100,'千':1000,'万':10000}
#将文本都转化成数字格式
def cn2dig(src):
    if src=="":
        return None
    m=re.match("\d+",src)
    if m:#如果匹配的都是数字，那就直接转化返回
        return int(m.group(0))
    rsl=0
    unit=1
    #倒序循环
    for item in src[::-1]:
        #如果查找的item在UTIL_CN_UNIT中，直接对unit赋值
        if item in UTIL_CN_UNIT.keys():
            unit=UTIL_CN_UNIT[item]
        #如果在UTIL_CN_NUM中，取出数字然后和unit相乘赋值给rsl
        elif item in UTIL_CN_NUM.keys():
            num=UTIL_CN_NUM[item]
            rsl+=num*unit
        else:
            return None
    if rsl<unit:
        rsl+=unit
    return rsl
#处理年份
def year2dig(year):
    res=''
    for item in year:
        #如果在字典中，就直接转化成数字然后拼接
        if item in UTIL_CN_NUM.keys():
            res=res+str(UTIL_CN_NUM[item])
        #否则就直接相加
        else:
            res=res+item
    m=re.match('\d+',res)
    if m:
        #判断如果年份是2位数，比如21年，就用当前日期的年份/100得到年份的前两位，然后再加上这两位数，就得到了年份
        if len(m.group(0))==2:
            return int(datetime.datetime.today().year/100)*100+int(m.group(0))
        else:
            return int(m.group(0))
    else:
        return None

In [53]:
text='我要住到明天下午三点三十五分'
print(text,time_extract(text),sep=':')

我要住到明天下午三点三十五分:['2021-03-04 15:35:00']
